In [13]:
%matplotlib inline
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sklearn
import os
import csv

Importing Maori lexicon

http://nzetc.victoria.ac.nz/tm/scholarly/tei-legalMaoriCorpus.html#legal_maori_lexicon.

http://www.maori.info/maori_language.htm

http://tereomaori.tki.org.nz/Teacher-tools/Te-Whakaipurangi-Rauemi/High-frequency-word-lists

Removed macrons, parentheses etc from file in Sublime Text.

In [141]:
maori_lexicon = set([])

with open('../data/raw/maori_legal_lexicon.csv', 'rb') as f:
    reader = csv.reader(f)
    rough_list = list(reader)
    for item in rough_list:
        for word in item:
            if (word!='' and ~word.isdigit()):
                for i in word.split(' '):
                    maori_lexicon.add(i.lower())

with open('../data/raw/maori_lexicon.csv', 'rb') as f:
    reader = csv.reader(f)
    rough_list = list(reader)
    for item in rough_list:
        for word in item:
            maori_lexicon.add(word.lower())

maori_lexicon.remove("")

Using a slightly edited verison of the first ~2000 from https://github.com/first20hours/google-10000-english/blob/master/20k.txt

http://www.quietaffiliate.com/free-first-name-and-last-name-databases-csv-and-sql/

In [248]:
english_lexicon = set([])

with open('../data/raw/english_lexicon_top_2000.csv', 'rb') as f:
    reader = csv.reader(f)
    rough_list = list(reader)
    for wordlist in rough_list:
        for word in wordlist:
            english_lexicon.add(word.lower())
            
with open('../data/raw/CSV_Database_of_First_And_Last_Names/CSV_Database_of_First_Names.csv', 'rU') as f:
    reader = csv.reader(f)
    rough_list = list(reader)
    for wordlist in rough_list:
        for word in wordlist:
            english_lexicon.add(word.lower())

with open('../data/raw/CSV_Database_of_First_And_Last_Names/CSV_Database_of_Last_Names.csv', 'rU') as f:
    reader = csv.reader(f)
    rough_list = list(reader)
    for wordlist in rough_list:
        for word in wordlist:
            english_lexicon.add(word.lower())                                
            
english_lexicon = english_lexicon.difference(maori_lexicon)

In [249]:
print len(english_lexicon)

93040


In [147]:
def word2features(word,maori=True):
    """Returns a word length normalized vector of all letter frequencies and bigrams.
       First entry is word, second is whether it is a Maori word or not."""
    if maori == True:
        features = [word,1]
    else:
        features = [word,0]
    word_length = len(word)
    alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l',
                'm','n','o','p','q','r','s','t','u','v','w','y','z']
    for letter in alphabet:
        features.append(word.count(letter)/float(word_length))
    for first_letter in alphabet:
        for second_letter in alphabet:
            features.append(word.count(first_letter+second_letter)/float(word_length))
    return features

In [148]:
words = []
for word in maori_lexicon:
    words.append(word2features(word))
for word in english_lexicon:
    words.append(word2features(word,maori=False))

In [149]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l',
            'm','n','o','p','q','r','s','t','u','v','w','y','z']
column_names = ['word', 'maori_binary']
for letter in alphabet:
    column_names.append(letter)
for first_letter in alphabet:
    for second_letter in alphabet:
        column_names.append(first_letter+second_letter)

In [150]:
words_df = pd.DataFrame(words)
words_df.columns = column_names
print words_df.head()

           word  maori_binary         a    b    c    d         e    f  \
0         ainga             1  0.400000  0.0  0.0  0.0  0.000000  0.0   
1       kahore              1  0.142857  0.0  0.0  0.0  0.142857  0.0   
2         putua             1  0.200000  0.0  0.0  0.0  0.000000  0.0   
3        takahe             1  0.333333  0.0  0.0  0.0  0.166667  0.0   
4  whakaminenga             1  0.250000  0.0  0.0  0.0  0.083333  0.0   

          g         h ...    zp   zq   zr   zs   zt   zu   zv   zw   zy   zz  
0  0.200000  0.000000 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.000000  0.142857 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.000000  0.000000 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.000000  0.166667 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.083333  0.083333 ...   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 652 columns]


In [265]:
X = words_df.drop(['word','maori_binary'],axis=1).values
y = words_df['maori_binary']

In [264]:
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import confusion_matrix

In [269]:
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)
cm = np.array([[0,0],[0,0]])
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    rus = RandomUnderSampler(ratio=0.3)
    X_train, y_train = rus.fit_sample(X_train,y_train)
    ros = RandomOverSampler()
    X_train, y_train = ros.fit_sample(X_train,y_train)
    model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                              silent=True, objective='binary:logistic', nthread=-1, \
                              gamma=0, min_child_weight=1, max_delta_step=0, \
                              subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                              base_score=0.5, missing=None)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    cm += confusion_matrix(y_test, y_pred)
print cm

[[88387  4654]
 [   13  2620]]


In [160]:
final_model = xgb.XGBClassifier(max_depth=5, learning_rate=0.1, n_estimators=100, \
                              silent=True, objective='binary:logistic', nthread=-1, \
                              gamma=0, min_child_weight=1, max_delta_step=0, \
                              subsample=1, colsample_bytree=1, colsample_bylevel=1, \
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, \
                              base_score=0.5, missing=None)
rus = RandomUnderSampler()
X_train, y_train = rus.fit_sample(X,y)
ros = RandomOverSampler()
X_train, y_train = ros.fit_sample(X_train,y_train)
final_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

Which words does the model get wrong?

In [161]:
maori_misclassified = []

for word in maori_lexicon:
    test=[]
    test.append(word2features(word))
    test_df = pd.DataFrame(test)
    binary = final_model.predict(test_df.drop([0,1],axis=1).values)
    if binary == 0:
        maori_misclassified.append(word)

print maori_misclassified

english_misclassified = []

for word in english_lexicon:
    test=[]
    test.append(word2features(word))
    test_df = pd.DataFrame(test)
    binary = final_model.predict(test_df.drop([0,1],axis=1).values)
    if binary == 1:
        english_misclassified.append(word)

print english_misclassified

['adze ', 'pl.', 'ripene', 'act', 'romalpa', 'henera']


KeyboardInterrupt: 

In [261]:
print english_misclassified

['petrak', 'higman', 'uptmor', 'urquhart', 'phang', 'wang', 'wank', 'wann', 'want', 'kraig', 'wrong', 'wrona', 'knapik', 'kuen', 'ingrim', 'nanik', 'nania', 'kown', 'gata', 'peetoom', 'unknow', 'hiatt', 'ehrke', 'magano', 'magana', 'totman', 'pinon', 'irr', 'tippin', 'tippie', 'nature', 'kamke', 'koehring', 'tonini', 'arkin', 'hepa', 'kohn', 'kohm', 'kohr', 'hone', 'parreira', 'hong', 'whapham', 'noxon', 'hai', 'ham', 'orrantia', 'hae', 'haq', 'har', 'haw', 'trongone', 'kamaka', 'raper', 'iwaoka', 'hopton', 'ingemi', 'okinaka', 'hawken', 'hawker', 'imming', 'minute', 'reining', 'honour', 'riemer', 'koppa', 'hegna', 'miehe', 'amarian', 'majeau', 'poaipuni', 'krain', 'agamao', 'potier', 'harteau', 'kreitner', 'gutman', 'tamaro', 'tamara', 'rehak', 'martineau', 'pepitone', 'houpe', 'houpt', 'trinka', 'weiher', 'kimura', 'koerner', 'otta', 'ramoutar', 'paniagua', 'kominek', 'kira', 'arrow', 'auter', 'auten', 'heenan', 'witt', 'with', 'tripi', 'thora', 'romito', 'maximum', 'nora', 'nork', '

Coding a classifier for road names. Since the road names contain numbers, spaces, hyphens and uppercase letters there's a bit of cleaning to be done on each road name.
1. Each roadname is split into a list of words, if there's only one word then it's a list containing only the word.
2. A series of tests are applied to each word in the list, if any of the indicators are that it's a Maori word then the function returns True, otherwise it continues on through each word in the name until it either finds a Maori word or runs out of words and returns False.

The ultimate aim here is to use the word classifier as little as possible. While it works fairly well there are other methods I'd rather use first (eg does the word contain a character which doesn't appear in the Maori alphabet).

In [268]:
def classify_as_maori(word):
    word = str(word)
    wordlist = word.lower().replace('-',' ').split()
    non_maori_letters = set(['b','c','d','f','j','l','q','s','v','x','y','z'])
    for word in wordlist:
        if word in maori_lexicon:
            return True
        elif word in english_lexicon:
            pass
        elif any((char in non_maori_letters) for char in word):
            pass
        elif word.isdigit():
            pass
        elif final_model.predict(pd.DataFrame([word2features(word)]).drop([0,1],axis=1).values)==1:
            return True
    return False

def classify_as_maori_diagnose(word):
    word = str(word)
    wordlist = word.lower().replace('-',' ').split()
    non_maori_letters = set(['b','c','d','f','j','l','q','s','v','x','y','z'])
    for word in wordlist:
        if word in maori_lexicon:
            print word,1
            return True
        elif word in english_lexicon:
            print word,2
            pass
        elif any((char in non_maori_letters) for char in word):
            print word,3
            pass
        elif word.isdigit():
            print word,4
            pass
        elif final_model.predict(pd.DataFrame([word2features(word)]).drop([0,1],axis=1).values)==1:
            print word,5
            return True
    return False

In [257]:
classify_as_maori_diagnose('twin')

twin 4


True

In [230]:
def test(x):
    for f in x:
        if f == 3:
            return True
        elif f == 4:
            pass
        elif f%2==0:
            return False
    return False

test([4,3])

True

# NZ Road Names

Downloaded the NZ Roads Addressing database https://data.linz.govt.nz

In [270]:
nz_roads_df = pd.read_csv('../data/raw/lds-nz-roads-addressing-CSV/nz-roads-addressing.csv')
nz_roads_df.head()

,WKT,road_id,geometry_class,road_type,road_name_class,full_road_name,road_name_label,road_name_prefix,road_name_body,road_name_type,road_name_suffix,route_name_body,route_name_number,route_name_alpha,route_name_suffix,full_road_name_ascii,road_name_label_ascii,road_name_body_ascii
0,MULTILINESTRING ((170.539440266700012 -45.8829...,1770045,Addressing Road,Unknown,Road,Harbour Heights,Harbour Hts,NaN,Harbour,Heights,NaN,NaN,NaN,NaN,NaN,Harbour Heights,Harbour Hts,Harbour
1,MULTILINESTRING ((170.359735333299994 -45.8823...,1770050,Addressing Road,Unknown,Road,Stanley Square,Stanley Sq,NaN,Stanley,Square,NaN,NaN,NaN,NaN,NaN,Stanley Square,Stanley Sq,Stanley
2,MULTILINESTRING ((169.532780550000012 -45.5881...,1770085,Addressing Road,Unknown,Road,Lake Onslow Road,Lake Onslow Rd,NaN,Lake Onslow,Road,NaN,NaN,NaN,NaN,NaN,Lake Onslow Road,Lake Onslow Rd,Lake Onslow
3,MULTILINESTRING ((170.490824783300013 -45.8825...,1770130,Addressing Road,Unknown,Road,William Street,William St,NaN,William,Street,NaN,NaN,NaN,NaN,NaN,William Street,William St,William
4,MULTILINESTRING ((170.478778566311604 -45.4729...,1770136,Addressing Road,Unknown,Road,Ainges Road,Ainges Rd,NaN,Ainges,Road,NaN,NaN,NaN,NaN,NaN,Ainges Road,Ainges Rd,Ainges


In [271]:
nz_roads_df['maori'] = nz_roads_df['road_name_body_ascii'].apply(classify_as_maori)

In [258]:
road_names = list(nz_roads_df['road_name_body_ascii'].values)
maori_roads = []
non_maori_roads = []
i=0
for road in road_names:
    if classify_as_maori(road):
        maori_roads.append(road)
    else:
        non_maori_roads.append(road)
print len(maori_roads)
print len(non_maori_roads)

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
13642
60952


In [259]:
print non_maori_roads

['Harbour', 'Stanley', 'Lake Onslow', 'William', 'Ainges', 'Centre', 'Oxford', 'Orwell', 'Clare', 'Crawford', 'Birsa', 'Waynestown', 'Muir', 'Mill', 'Tydvil', 'Troup', 'Lime', 'Liverpool', 'Lincoln', 'Long', 'Lewis', 'Lighthouse', 'Rattray', 'Rees', 'Simm', 'Short', 'Northampton', 'Ascot', 'Connell', 'Bulleid', 'Meek', 'McCulloch', 'McLeods', 'McCulloch', 'Fitzroy', 'Finlay', 'Hart', 'Simmonds', 'Shennans', 'Fastnet', 'Shale', 'Jus', 'Cobb', 'Stafford', 'Fraser', 'Railway Station', 'Enwright', 'Howell', 'McCready', 'Cheviot', 'Ferry', 'Melrose', 'Jedburgh', 'Loop', 'Sullivan', 'Stafford', 'Clouston', 'Cemetery', 'Lewis', 'Lethbridge', 'Sandflat', 'Schoolhouse', 'Rutherford', 'White', 'Wilton', 'Chester', 'Amisfield', 'Vincent', 'Upper Manorburn', 'Hertford', 'Kilmarnoch', 'Faulks', 'Mulberry', 'Coronation', 'Collins', 'Kelvin', 'Bowen', 'Boundary', 'Butement', 'Tallisker', 'Sydney', 'Bay View', 'Beaumont', 'Beech', 'Beach', 'Barra', 'Wattle', 'Warren', 'Watts', 'Mackay', 'Dublin', 'Dom

In [260]:
print maori_roads

['Totara Peak', 'Totara', 'Waiareka', 'Waiareka', 'Maori Crossing', 'Arapeta', 'Upper Taieri-Paerau', 'Waipiata-Kyeburn', 'Hermitage', 'Eureka', 'Kawarau', 'Wakatipu', 'Rata', 'Te Ra', 'Matai', 'Marewa', 'Orokonui', 'Pari', 'Poto', 'Rawhiti', 'Totara', 'Taratu', 'Matuanui', 'Marama', 'Craggy Tor', 'Matau', 'Pukeawa', 'Taine', 'Minaret', 'Otokia', 'Muriwai', 'Waimea', 'Utah', 'Toi Toi', 'Kauana', 'Marinui', 'Aurum', 'Waimatuku South', 'Waikouro Shortcut', 'Rimu', 'Moana', 'Tainui', 'Kaka', 'Oware', 'Matai', 'Paringa', 'Tipapa', 'Wiremu', 'Tau', 'Weka', 'Orari', 'Ohapi Settlement', 'Karaka', 'Miro', 'Rata', 'Iri Irikapua', 'Te Puke', 'Rima', 'Rehua', 'Tekoa', 'Muritai', 'Monkton', 'Waitaki', 'Waratah', 'Ngaio', 'Tauiwi', 'Ruakaka', 'Huia', 'Moorpark', 'Maori Reserve', 'Waiora', 'Kanuka', 'Kaikanui', 'Paparua', 'Tahi Rua', 'Kotare', 'Ophir', 'Waimea', 'Omaru', 'Nikau', 'Konini', 'Tawiri', 'Manuka', 'Tonga', 'Turiwhaia', 'Mahuri', 'Kupe', 'Utopia', 'Koromiko', 'Kowhai', 'Maire', 'Matai', '